In [1]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import matplotlib.pyplot as plt
import warnings
import re
import pandas as pd, numpy as np, string
from sklearn_pandas import DataFrameMapper, cross_val_score
warnings.filterwarnings('ignore')

In [24]:
dataset = pd.read_csv('dataset/good_reads_final.csv')


In [25]:
dataset.columns

Index(['author_average_rating', 'author_gender', 'author_genres', 'author_id',
       'author_name', 'author_page_url', 'author_rating_count',
       'author_review_count', 'birthplace', 'book_average_rating',
       'book_fullurl', 'book_id', 'book_title', 'genre_1', 'genre_2',
       'num_ratings', 'num_reviews', 'pages', 'publish_date', 'score'],
      dtype='object')

In [26]:
# y = dataset[['']]
dataset = dataset[['author_average_rating', 'author_gender', 'author_genres', 'author_rating_count', 'author_review_count', 'birthplace', 'book_average_rating', 'book_id', 'book_title', 'genre_1', 'genre_2', 'num_ratings', 'num_reviews', 'pages', 'publish_date', 'score']]


##### Página URL do Autor(a) foi retirado também devido a conter o nome dos Autores(as)
##### URL do Livro também foi retirada devido a trazer apenas informações que podem ser derivadas das outras *features*
##### O ano de publicação também foi retirado devido a inconsistencia da tabela (um número consideravel célulaapresentavam (?)


In [27]:
# dataset.head(5)

In [28]:
# (Jan(uary)?|Feb(ruary)?|Mar(ch)?|Apr(il)?|May|Jun(e)?|Jul(y)?|Aug(ust)?|Sep(tember)?|Oct(ober)?|Nov(ember)?|Dec(ember)?)((\s*\d{1,2}(th|st|rd|nd)|(\s+\d{1,2},?)|))

In [29]:
dataset = dataset.replace(to_replace=r'(Jan(uary)?|Feb(ruary)?|Mar(ch)?|Apr(il)?|May|Jun(e)?|Jul(y)?|Aug(ust)?|Sep(tember)?|Oct(ober)?|Nov(ember)?|Dec(ember)?)', value='', regex=True)

In [30]:
# dataset.head(5)

In [31]:
dataset = dataset.replace(to_replace=r'(\d{1,2}(th|nd|st|rd))', value='', regex=True)
# dataset = dataset.replace(to_replace=r'(\bby\b)', value=None, regex=Tr

In [32]:
# dataset.head(5)

In [33]:
# dataset['publish_date'] = dataset['publish_date'].astype(int)
print(dataset.index)

RangeIndex(start=0, stop=22891, step=1)


In [34]:
dataset = dataset[dataset.publish_date != 'by']


In [35]:
# print(dataset.index)

In [36]:
# dataset.dtypes

In [50]:
# print(dataset[dataset['publish_date'].isnull()])
dataset['publish_date'] = pd.to_numeric(dataset['publish_date'], errors='coerce')
dataset =dataset.dropna(subset=['publish_date'])
dataset['publish_date'] = dataset['publish_date'].astype(int)
dataset['author_genres'] = dataset['author_genres'].astype(str)
dataset['birthplace'] = dataset['birthplace'].astype(str)
dataset['book_title'] = dataset['book_title'].astype(str)
dataset['genre_1'] = dataset['genre_1'].astype(str)
dataset['genre_2'] = dataset['genre_2'].astype(str)



In [38]:
# dataset.dtypes

In [51]:
dataset['author_average_rating'] = np.where(dataset['author_average_rating'] > 100.0, dataset['author_average_rating']/100, dataset['author_average_rating'])
dataset['book_average_rating'] = np.where(dataset['book_average_rating'] > 100.0, dataset['book_average_rating']/100, dataset['book_average_rating'])


In [52]:
# dataset.tail(5)

In [59]:
mapper = DataFrameMapper([(['author_average_rating', 'author_rating_count', 'author_review_count', 'book_average_rating', 'num_ratings', 'num_reviews', 'pages', 'score'], None),
                          (['publish_date'], preprocessing.StandardScaler()),
                          (['author_gender'], preprocessing.LabelEncoder())

], df_out=True)

In [60]:
mapper

DataFrameMapper(default=False, df_out=True,
                features=[(['author_average_rating', 'author_rating_count',
                            'author_review_count', 'book_average_rating',
                            'num_ratings', 'num_reviews', 'pages', 'score'],
                           None),
                          (['publish_date'],
                           StandardScaler(copy=True, with_mean=True,
                                          with_std=True)),
                          (['author_gender'], LabelEncoder())],
                input_df=False, sparse=False)

In [61]:
df = mapper.fit_transform(dataset.copy())

In [62]:
df

,author_average_rating_author_rating_count_author_review_count_book_average_rating_num_ratings_num_reviews_pages_score_0,author_average_rating_author_rating_count_author_review_count_book_average_rating_num_ratings_num_reviews_pages_score_1,author_average_rating_author_rating_count_author_review_count_book_average_rating_num_ratings_num_reviews_pages_score_2,author_average_rating_author_rating_count_author_review_count_book_average_rating_num_ratings_num_reviews_pages_score_3,author_average_rating_author_rating_count_author_review_count_book_average_rating_num_ratings_num_reviews_pages_score_4,author_average_rating_author_rating_count_author_review_count_book_average_rating_num_ratings_num_reviews_pages_score_5,author_average_rating_author_rating_count_author_review_count_book_average_rating_num_ratings_num_reviews_pages_score_6,author_average_rating_author_rating_count_author_review_count_book_average_rating_num_ratings_num_reviews_pages_score_7,publish_date,author_gender
0,4.01,74399.0,6268.0,4.02,5260.0,375.0,277.0,3230.0,0.073614,0
1,4.15,3726435.0,142704.0,4.13,2229163.0,65227.0,465.0,3062.0,0.110497,1
2,4.00,76496.0,7975.0,3.99,2151.0,391.0,354.0,4585.0,0.191640,0
3,3.88,5522.0,489.0,4.14,1844.0,173.0,438.0,1533.0,-0.147686,1
4,4.10,291013.0,13453.0,4.01,17051.0,1890.0,326.0,2105.0,-0.014906,0
...,...,...,...,...,...,...,...,...,...,...
22886,3.94,52944.0,4558.0,3.82,430.0,33.0,72.0,1215.0,0.184263,0
22887,4.29,902.0,428.0,4.56,82.0,57.0,342.0,1321.0,0.176887,1
22888,4.00,15626.0,3530.0,3.73,2698.0,375.0,237.0,1679.0,0.176887,0
22889,4.20,4689.0,828.0,3.89,356.0,38.0,222.0,1897.0,0.162133,0


In [ ]:
# X = dataset[['author_average_rating', 'author_rating_count', 'author_review_count', 'book_average_rating', 'num_ratings', 'num_reviews', 'pages', 'score']]
# X_to_text = dataset[['author_genres', 'birthplace', 'book_title','genre_1', 'genre_2']]
# # X_to_bynary=['publish_date']


In [ ]:
# year_binarized = pd.DataFrame(preprocessing.LabelBinarizer().fit_transform(dataset.publish_date))
# X = pd.concat([X,year_binarized],axis=1)
# X.head(5)

In [ ]:
encoder = preprocessing.LabelEncoder()
y = encoder.fit_transform(y)
# valid_y = encoder.fit_transform(valid_y)

In [ ]:
def countVec(data, sw=None, sa=None, lc=True):
    count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}', stop_words=sw, strip_accents= sa, lowercase=lc)
    count_vect.fit(data)
    x_count =  count_vect.transform(data)
#     xvalid_count =  count_vect.transform(valid_x)
    return x_count

In [ ]:
def tfidfVec(data, sw=None, sa=None, lc=True):
    tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000, stop_words=sw, strip_accents= sa, lowercase=lc)
    tfidf_vect.fit(data)
    xtrain_tfidf =  tfidf_vect.transform(train_x)
    xvalid_tfidf =  tfidf_vect.transform(valid_x)
    return xtrain_tfidf, xvalid_tfidf

In [ ]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    lista = []
    lista.append(metrics.accuracy_score(valid_y, predictions))
    lista.append(metrics.recall_score(valid_y, predictions))
    lista.append(metrics.f1_score(valid_y, predictions))
    lista.append(metrics.confusion_matrix(valid_y, predictions))
    
    return lista

In [ ]:
def plotMetrics(listOfResults, listOfMetrics):

    for l in range(len(listOfResults)):
        ax = plt.axes()
        ax.bar(l, listOfResults[l], label=listOfMetrics[l], align='center', width=0.4)

    # fig = plt.figure(figsize=(1,1))
    plt.title("Análise de Gênero do Autor(a) de acordo com o titulo da obra")
    plt.legend(listOfMetrics)
    plt.ylabel("Porcentagem")
    plt.xlabel("Métricas")
    plt.xticks(range(len(listOfResults)))
    plt.show()

In [ ]:
def plot_confusion_matrix(classes, cm,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Matrix de Confusão Normalizada'
        else:
            title = 'Matrix de Confusão'

    # Compute confusion matrix
    cm = cm
    # Only use the labels that appear in the data
    classes = classes
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Matrix de Confusão Normalizada")
    else:
        print('Matrix de Confusão')

#     print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='Rotulo Verdadeiro',
           xlabel='Rotulo Predito')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            if i == 0:
                ax.text(j, i+0.2, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
            else:
                ax.text(j, i-0.2, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax


In [ ]:
models = [naive_bayes.MultinomialNB(), linear_model.LogisticRegression(), ensemble.RandomForestClassifier()]
features = [['english', None, True], ['english', 'ascii', True], 
            ['english', None, False], ['english', 'ascii', False], 
            [None, 'ascii', True], [None, 'ascii', False]]

# Para Bag of Words

In [ ]:
x_count = countVec(X_to_text, None, 'ascii', False)
X = pd.concat([X,x_count],axis=1)

train_x, valid_x, train_y, valid_y = model_selection.train_test_split(X,y, test_size = 0.2)

l = train_model(md, xtrain_count, train_y, xvalid_count)

In [ ]:
d = {}
l = []
i = 0
for md in models:
    l.append(md)
    for f in features:
        x_count = countVec(X_to_text, f[0], f[1], f[2])
        X = pd.concat([X,x_count],axis=1)

        train_x, valid_x, train_y, valid_y = model_selection.train_test_split(X,y, test_size = 0.2)

        l.append(train_model(md, xtrain_count, train_y, xvalid_count))
    d[i] = l
    i+=1
    l = []


In [ ]:
for item in d.items():
    print(item, '\n')

### Com o print acima, podemos concluir que utilizando o método Bag of Words e os três algoritmos:

* Que o melhor cenário é não por todas as palavras em caixa baixa com remoção de caracteres não ASCII e sem remoção de stopwords;
* A remoção de StopWords, nesse caso apenas atrapalha o desempenho do algoritmo;
* Já a remoção de caracteres "não ascii" mostrou-se vantajosa;
* O algoritmo de Floresta Randômica, foi o que se saiu melhor neste experimento.

# Para TFIDF

In [ ]:
d2 = {}
l2 = []
i = 0
for md in models:
    l2.append(md)
    for f in features:
        xtrain_tfidf, xvalid_tfidf = tfidfVec(dataset['book_title'], f[0], f[1], f[2])
        l2.append(train_model(md, xtrain_tfidf, train_y, xvalid_tfidf))
    d2[i] = l2
    i+=1
    l2 = []

In [ ]:
for item in d2.items():
    print(item, '\n')

### Com o print acima, podemos concluir que utilizando o método TFIDF e os três algoritmos:

* Que o melhor cenário é não por todas as palavras em caixa baixa com remoção de caracteres não ASCII e sem remoção de stopwords;
* A remoção de StopWords, nesse caso apenas atrapalha o desempenho do algoritmo;
* Já a remoção de caracteres "não ascii" mostrou-se vantajosa;
* Mas uma vez o algoritmo de Floresta Randômica, foi o que se saiu melhor neste experimento.
* Podemos observar também que o uso da técnica TFIDF, mostrou-se obter resultados piores do que uma técnica mais simples como o Bag of Words

# Análise Preliminar dos Resultados


### Para a análise dos resultados será considerada a execução com os seguintes parâmetros:

* Sem aplicação de *lowercase*
* Com remoção de caracteres "não ASCII"
* Sem remoção de *stopwords*
* Com a aplicação da técnica *bag of words*

#### Os resultados são compostos das seguintes métricas, e seus resultados:

* Acurácia: 0,73
* Recall:   0,82
* F1 Score: 0,77

Apesar de acertar aproximadamente o gênero de 7 autores(as) a cada 10 vez, o recall do algoritmo é consideravelmente alto, o que significa que consegue identificar com uma boa margem aqueles o qual ele selecionou como sendo de cada gênero.
Todavia, observando a F1 Score, podemos inferir que a precisão do algoritmo é bem inferior ao recall, o que faz com que muitos titulos tenham sido atribuídos ao gênero errado.

In [ ]:
listOfResults = [d2[2][6][0], d2[2][6][1], d2[2][6][2]]
listOfMetrics = ['Acuracia', 'Recall', 'F1 Score']
plotMetrics(listOfResults, listOfMetrics)

In [ ]:
np.set_printoptions(precision=2)
cl = ['Homem', 'Mulher']
cl = np.asarray(cl)
plot_confusion_matrix(cl, d[2][6][3])
plot_confusion_matrix(cl, d[2][6][3], normalize=True)
plt.show()

### Com base na Matriz de Confusão, pode-se observar que o Algoritmo tem uma maior acerto (cerca de 20% a mais) quando identifica as autoras, logo ele classifica muito livro que é escrito por um autor, como sendo escrito por uma autora.